# Build a Voice-Powered Kitchen Assistant with Google ADK and Elastic Agent Builder - Setup

Learn to build a voice assistant that queries Elasticsearch using Google ADK and Agent Builder. Covers MCP protocol, LiveAPI voice streaming and semantic search.

## Install Dependencies

Install the required Python packages for Elasticsearch connectivity and environment variable management.

In [25]:
!pip install elasticsearch dotenv -q

## Configuration

Create a `.env` file in this directory with your credentials:

```
ES_ENDPOINT=
KIBANA_ENDPOINT=
ES_API_KEY=
```

## Import Libraries

Import all necessary libraries for working with Elasticsearch, handling JSON data, and making HTTP requests to the Kibana API.

In [39]:
import json
import requests
from dotenv import load_dotenv
import os
from elasticsearch import Elasticsearch
from elasticsearch import helpers

load_dotenv()

True

## Declare Variables and Initialize Elasticsearch Client

Set up environment variables and create the Elasticsearch client. Also define the index name and inference endpoint ID that will be used throughout the notebook.

In [40]:
ES_ENDPOINT = os.getenv("ES_ENDPOINT")
KIBANA_ENDPOINT = os.getenv("KIBANA_ENDPOINT")
ES_API_KEY = os.getenv("ES_API_KEY")

es_client = Elasticsearch(ES_ENDPOINT, api_key=ES_API_KEY)

INDEX_NAME = "cooking-recipes"
INFERENCE_ID = "jina-embeddings"

KIBANA_HEADERS = {
    "Authorization": f"ApiKey {ES_API_KEY}",
    "Content-Type": "application/json",
    "kbn-xsrf": "true",
}

## Create Inference Endpoint

Create a text embedding inference endpoint using Jina Embeddings v3, a high-quality retrieval model provided by Elastic. This endpoint will be used for semantic search capabilities in our recipe index.

In [41]:
try:
    # Create inference endpoint with Jina embeddings v3
    inference_config = {
        "service": "elastic",
        "service_settings": {"model_id": "jina-embeddings-v3"},
    }

    es_client.inference.put(
        task_type="text_embedding", inference_id=INFERENCE_ID, body=inference_config
    )

    print(f"✅ Inference endpoint '{INFERENCE_ID}' created successfully")
except Exception as e:
    print(f"❌ Error creating inference endpoint: {e}")
    raise

✅ Inference endpoint 'jina-embeddings' created successfully


## Create Knowledge Index

Create the `knowledge` index with a mapping that includes:
- **Text fields**: `name`, `ingredients`, `procedure` - for full-text search
- **Keyword fields**: `allergens`, `category`, `dietary` - for filtering and aggregations
- **Semantic field**: Uses `semantic_text` type with Jina embeddings for semantic search

The `copy_to` property aggregates content from multiple fields into `semantic_field` for unified semantic search.

In [42]:
try:
    es_client.indices.exists(index=INDEX_NAME)

    knowledge_mapping = {
        "properties": {
            "name": {"type": "text", "copy_to": "semantic_field"},
            "ingredients": {"type": "text", "copy_to": "semantic_field"},
            "allergens": {"type": "keyword", "copy_to": "semantic_field"},
            "procedure": {"type": "text", "copy_to": "semantic_field"},
            "prep_time_minutes": {"type": "integer"},
            "category": {"type": "keyword", "copy_to": "semantic_field"},
            "dietary": {"type": "keyword", "copy_to": "semantic_field"},
            "semantic_field": {
                "type": "semantic_text",
                "inference_id": INFERENCE_ID,
            },
        }
    }

    es_client.indices.create(index=INDEX_NAME, mappings=knowledge_mapping)
except Exception as e:
    print(f"Error creating index: {e}")

## Ingest Recipe Data

Load recipe data from `dataset.json` and bulk index it into Elasticsearch. The bulk API is used for efficient ingestion of multiple documents.

In [43]:
def build_bulk_actions(documents, index_name):
    for doc in documents:
        yield {"_index": index_name, "_source": doc}


with open("dataset.json", "r") as f:
    knowledge_docs = json.load(f)

success, failed = helpers.bulk(
    es_client,
    build_bulk_actions(knowledge_docs, INDEX_NAME),
    refresh=True,
)
print(f"{success} documents indexed successfully")

12 documents indexed successfully


## Create Agent Builder Tool

Register a semantic search tool in Agent Builder via the Kibana API. This tool:
- **ID**: `recipe_semantic_search` - unique identifier used by the ADK agent
- **Type**: `index_search` - performs searches on Elasticsearch indices
- **Tags**: `semantic` - enables semantic search capabilities

The tool description guides the agent on when and how to use this tool for recipe queries.

In [44]:
recipe_search_tool = {
    "id": "recipe_semantic_search",
    "type": "index_search",
    "description": "Search kitchen recipes including ingredients, allergens, dietary restrictions, preparation procedures, and cooking times. Uses semantic search to find relevant recipes even without exact keyword matches.",
    "tags": ["semantic"],
    "configuration": {
        "pattern": INDEX_NAME,
    },
}

try:
    response = requests.post(
        f"{KIBANA_ENDPOINT}/api/agent_builder/tools",
        headers=KIBANA_HEADERS,
        json=recipe_search_tool,
    )

    if response.status_code == 200:
        print("✅ Recipe semantic search tool created successfully")
    elif response.status_code == 400:
        # Check if it's because the tool already exists
        if "already exists" in response.text.lower():
            print("ℹ️  Recipe search tool already exists, continuing...")
        else:
            print(
                f"⚠️  Failed to create tool: {response.status_code}. \n Response: {response.text}"
            )
    else:
        print(
            f"⚠️  Failed to create tool: {response.status_code}. \n Response: {response.text}"
        )
except Exception as e:
    print(f"❌ Error creating tool: {e}")

✅ Recipe semantic search tool created successfully


---

## Cleanup

Run the following cells to clean up all resources created in this notebook. This includes:
1. Deleting the Agent Builder tool
2. Deleting the Elasticsearch index
3. Deleting the inference endpoint

### Delete Agent Builder Tool

Remove the semantic search tool from Agent Builder using the Kibana API.

In [32]:
TOOL_ID = "recipe_semantic_search"

try:
    response = requests.delete(
        f"{KIBANA_ENDPOINT}/api/agent_builder/tools/{TOOL_ID}",
        headers=KIBANA_HEADERS,
    )

    if response.status_code == 200:
        print(f"✅ Agent Builder tool '{TOOL_ID}' deleted successfully")
    else:
        print(f"ℹ️  Tool '{TOOL_ID}' not found, may have already been deleted")
except Exception as e:
    print(f"❌ Error deleting tool: {e}")

ℹ️  Tool 'recipe_semantic_search' not found, may have already been deleted


### Delete Elasticsearch Index

Remove the knowledge index containing the recipe data.

In [37]:
try:
    es_client.indices.delete(index=INDEX_NAME)
    print(f"✅ Index '{INDEX_NAME}' deleted successfully")
except Exception as e:
    if "index_not_found_exception" in str(e):
        print(f"ℹ️  Index '{INDEX_NAME}' not found, may have already been deleted")
    else:
        print(f"❌ Error deleting index: {e}")

✅ Index 'cooking-recipes' deleted successfully


### Delete Inference Endpoint

Remove the Jina embeddings inference endpoint from Elasticsearch.

In [38]:
try:
    es_client.inference.delete(inference_id=INFERENCE_ID)
    print(f"✅ Inference endpoint '{INFERENCE_ID}' deleted successfully")
except Exception as e:
    if "resource_not_found_exception" in str(e):
        print(
            f"ℹ️  Inference endpoint '{INFERENCE_ID}' not found, may have already been deleted"
        )
    else:
        print(f"❌ Error deleting inference endpoint: {e}")

✅ Inference endpoint 'jina-embeddings' deleted successfully
